# [SPADE](https://spade-mas.readthedocs.io/en/latest/index.html): Smart Python Agent Development Environment 

# BOWLDI

The **BOWLDI** idea is to combine ontologies (OWL) and BDI agents. In particular, it is envisioned to work with the **SPADE** platform, i.e. the BDI implementation used by SPADE, which is based on the AgentSpeak language. Therefore, BOWLDI allows to use OWL ontologies in the BDI agents implemented in SPADE. This is achieved by translating OWL/RDF files into AgentSpeak code and vice versa.

The conversion is performed within the `BOWLDIConverter` Python class that uses `owlready2` library to read ontology files or store information into them. Inversely, the said class can also convert AgentSpeak code into OWL/RDF files. The conversion is based on the mapping of OWL classes and properties to AgentSpeak plans and beliefs.

The class is envisioned as a modular construct that supports various functionalities, each of which may be used on their own as well. The main functionalities are:
- **Ontology to AgentSpeak conversion**: The conversion of OWL ontologies into AgentSpeak code.
- **AgentSpeak to Ontology conversion**: The conversion of AgentSpeak code into OWL ontologies.

Using the `BOWLDIConverter` class can be as easy as creating an instance of the class and calling the desired method.

The implementation of the `BOWLDIConverter` class is publicly available on GitHub [repository](https://github.com/AILab-FOI/MAGO/blob/main/Extra/BOWLDI/bowldi.py).

You can download it e.g. by using `wget`.

In [ ]:
!wget https://raw.githubusercontent.com/AILab-FOI/MAGO/main/Extra/BOWLDI/bowldi.py

The only library necessary for `BOWLDIConverter` to work is `owlready2`. You can install it by running the following command:

In [ ]:
try:
    import owlready2
except:
    !pip install owlready2

Otherwise, you can create a new Python environment with the necessary libraries. One possible solution is to create a Conda environment using the following environment definition:

```yaml
name: bowldiclass

dependencies:
  - python
  - conda-forge::owlready2
  - conda-forge::pexpect
  - pip
  - pip:
    - spade==4.0.2
    - spade_bdi

# conda env create -f env.yml
# conda env update -f env.yml --prune
```

This file is also available publicly at GitHub [repository](https://github.com/AILab-FOI/MAGO/blob/main/Documents/250320%20Class/env.yml), and can be downloaded using `wget` as well:

```bash
wget https://raw.githubusercontent.com/AILab-FOI/MAGO/main/Documents/250320%20Class/env.yml
```

## Example 1: Converting AgentSpeak to OWL

`BOWLDIConverter` can take a string input of AgentSpeak code and convert it to an ontology file. The following is a simple example containing: a couple of beliefs. These can be mapped to:
- two ontology concepts (lines 5--7),
- one individual (line 10),
- two properties (lines 13 and 16).

In [ ]:
from bowldi import BOWLDIConverter

input_data = """
// Concepts and Hierarchies
concept(person).
concept(wizard).
is_a(wizard, person).

// Individual
individual(gandalf, wizard).

// Object Property
object_property(person, is_friend_with, person).

// Data Property
data_property(person, has name, string)."""

converter = BOWLDIConverter(
    input_data=input_data,
)

converter.get_response()

In [ ]:
!cat output.owl

The response received on line 22 should read:

```json
{
  "status": "success",
  "message": "Conversion complete.",
  "file": "<path to file>/output.owl"
}
```

Converting AgentSpeak to an ontology file will always yield a file output, i.e. will always write the output to a file. The generated file `output.owl` should have the following content, based on the input AgentSpeak code:

```xml
<?xml version="1.0"?>
<rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
    xmlns:xsd="http://www.w3.org/2001/XMLSchema#"
    xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
    xmlns:owl="http://www.w3.org/2002/07/owl#" xml:base="file://<path to file>/output.owl"
    xmlns="file://<path to file>/output.owl#">

    <owl:Ontology rdf:about="file://<path to file>/output.owl"/>

    <owl:ObjectProperty rdf:about="#is_friend_with">
        <rdfs:domain rdf:resource="#person"/>
        <rdfs:range rdf:resource="#person"/>
        <rdfs:label xml:lang="en-gb">is friend with</rdfs:label>
        <rdfs:isDefinedBy rdf:datatype="http://www.w3.org/2001/XMLSchema#string">self</rdfs:isDefinedBy>
    </owl:ObjectProperty>

    <owl:DatatypeProperty rdf:about="#has_name">
        <rdfs:domain rdf:resource="#person"/>
        <rdfs:range rdf:resource="http://www.w3.org/2001/XMLSchema#string"/>
        <rdfs:label xml:lang="en-gb">has name</rdfs:label>
        <rdfs:isDefinedBy rdf:datatype="http://www.w3.org/2001/XMLSchema#string">self</rdfs:isDefinedBy>
    </owl:DatatypeProperty>

    <owl:Class rdf:about="#person">
        <rdfs:subClassOf rdf:resource="http://www.w3.org/2002/07/owl#Thing"/>
        <rdfs:label xml:lang="en-gb">person</rdfs:label>
        <rdfs:isDefinedBy rdf:datatype="http://www.w3.org/2001/XMLSchema#string">self</rdfs:isDefinedBy>
    </owl:Class>

    <owl:Class rdf:about="#wizard">
        <rdfs:subClassOf rdf:resource="#person"/>
        <rdfs:label xml:lang="en-gb">wizard</rdfs:label>
        <rdfs:isDefinedBy rdf:datatype="http://www.w3.org/2001/XMLSchema#string">self</rdfs:isDefinedBy>
    </owl:Class>

    <owl:NamedIndividual rdf:about="#gandalf">
        <rdf:type rdf:resource="#wizard"/>
        <rdfs:label xml:lang="en-gb">gandalf</rdfs:label>
        <rdfs:isDefinedBy rdf:datatype="http://www.w3.org/2001/XMLSchema#string">self</rdfs:isDefinedBy>
    </owl:NamedIndividual>

</rdf:RDF>
```

## Example 2: Converting OWL to AgentSpeak

`BOWLDIConverter` class can take an ontology file and convert it to AgentSpeak code. The following is a simple example of reading the ontology created in the previous example and converting it to AgentSpeak code.

In [ ]:
from bowldi import BOWLDIConverter

converter = BOWLDIConverter(
    input_data_path="output.owl",
    output_data_path="example_output.asl",
)

if converter.get_response().get("status") == "success":
    print(converter.agentspeak_output)

In [ ]:
!cat example_output.asl

The above code will take the ontology stored in the file `output.owl` and convert it to AgentSpeak code. The output is saved in the provided `output_data_path` file. In addition to receiving the output saved into the given file, the output can also be retrieved as a string using the `agentspeak_output` attribute.

The rendered AgentSpeak code should look as follows, based on the expected input:

```prolog
concept(person)[source(['self'])].
concept(wizard)[source(['self'])].
is_a(wizard, person)[source(['self'])].
object_property(person, is_friend_with, person)[source(['self'])].
data_property(person, has_name, str)[source(['self'])].
individual(gandalf, wizard)[source(['self'])].
```

This last output features a list of beliefs, but this time each of those is given a `source` attribute. This attribute should be used to tell the agent where the particular piece of knowledge comes from, i.e. from which agent or knowledge source (e.g. an ontology) it originates. In this instance, all the knowledge comes from the agent itself, hence the `self` value.

## Example 3: Converting OWL to AgentSpeak with an external source

A simple ontology has been prepared, featuring a couple of concepts and properties, related to the domain of the Lord of the Rings novel. The ontology is available publicly on GitHub [repository](https://github.com/AILab-FOI/MAGO/blob/main/Documents/250320%20Class/lotr_example.owl), and it can be visualized using WebOWL in this [link](https://service.tib.eu/webvowl/#iri=https://raw.githubusercontent.com/AILab-FOI/MAGO/refs/heads/main/Documents/250320%20Class/onto_example.rdf#)

The linked ontology contains the following information:

**Classes**

- `person`
- `wizard` (subclass of `person`)
- `elf` (subclass of `person`)
- `human` (subclass of `person`)
- `dwarf` (subclass of `person`)
- `hobbit` (subclass of `person`)
- `kingdom`
- `ring_of_power`

**Object Properties**

- `has_king` (domain: `kingdom`, range: `person`)
- `is_friend_with` (domain: `person`, range: `person`)
- `is_in_team_with` (domain: `person`, range: `person`)
- `has_ring` (domain: `person`, range: `ring_of_power`)
- `fights_against` (domain: `person`, range: `person`)

**Datatype Properties**

- `has_name` (domain: `person`, range: `string`)
- `description` (domain: `ring_of_power`, range: `string`)

Importing the above ontology, let us prepare another ontology that uses the concepts of the imported ontology to construct additional data. The developed example ontology that imports the `lotr_ontology.owl` is available [online](https://github.com/AILab-FOI/MAGO/blob/main/Documents/250320%20Class/onto_example.rdf) as well. It may be downloaded e.g. by using `wget`.

In [ ]:
!wget https://raw.githubusercontent.com/AILab-FOI/MAGO/refs/heads/main/Documents/250320%20Class/onto_example.rdf

This example ontology extends the `lotr_ontology.owl` and defines individuals of the classes stored therein.

The three defined individuals are described as follows, using the RDF/XML syntax generated by Protégé:

```xml
    <!-- <path to onto>/onto_example.rdf#OWLNamedIndividual_443b4123_3a46_49a2_b8c2_29ced3899779 -->

    <owl:NamedIndividual rdf:about="<path to onto>/onto_example.rdf#OWLNamedIndividual_443b4123_3a46_49a2_b8c2_29ced3899779">
        <rdf:type rdf:resource="<path to onto>/lotr_example.owl#wizard"/>
        <lotr:has_name>Gandalf the Grey</lotr:has_name>
        <lotr:has_name>Mithrandir</lotr:has_name>
        <rdfs:label xml:lang="en-gb">Gandalf the Grey</rdfs:label>
    </owl:NamedIndividual>
    <owl:Axiom>
        <owl:annotatedSource rdf:resource="<path to onto>/onto_example.rdf#OWLNamedIndividual_443b4123_3a46_49a2_b8c2_29ced3899779"/>
        <owl:annotatedProperty rdf:resource="<path to onto>/lotr_example.owl#has_name"/>
        <owl:annotatedTarget>Gandalf the Grey</owl:annotatedTarget>
        <rdfs:isDefinedBy>Bogdan</rdfs:isDefinedBy>
    </owl:Axiom>
    


    <!-- <path to onto>/onto_example.rdf#OWLNamedIndividual_8584c614_acbd_4080_9032_a3a6dd9d1a9b -->

    <owl:NamedIndividual rdf:about="<path to onto>/onto_example.rdf#OWLNamedIndividual_8584c614_acbd_4080_9032_a3a6dd9d1a9b">
        <rdf:type rdf:resource="<path to onto>/lotr_example.owl#human"/>
        <lotr:is_friend_with rdf:resource="<path to onto>/onto_example.rdf#OWLNamedIndividual_443b4123_3a46_49a2_b8c2_29ced3899779"/>
        <lotr:is_friend_with rdf:resource="<path to onto>/onto_example.rdf#OWLNamedIndividual_96afddcd_81ae_467a_9468_225b426210f4"/>
        <lotr:has_name>Aragorn, son of Arathorn</lotr:has_name>
        <rdfs:label xml:lang="en-gb">Aragorn, son of Arathorn</rdfs:label>
    </owl:NamedIndividual>
    


    <!-- <path to onto>/onto_example.rdf#OWLNamedIndividual_96afddcd_81ae_467a_9468_225b426210f4 -->

    <owl:NamedIndividual rdf:about="<path to onto>/onto_example.rdf#OWLNamedIndividual_96afddcd_81ae_467a_9468_225b426210f4">
        <rdf:type rdf:resource="<path to onto>/lotr_example.owl#elf"/>
        <lotr:has_name>Legolas Greenleaf</lotr:has_name>
        <rdfs:label xml:lang="en-gb">Legolas</rdfs:label>
    </owl:NamedIndividual>
```

When we use the `BOWLDIConvert` class to convert the information of `onto_example.rdf` into AgentSpeak code, the output includes a defined source value for every belief. Information that was defined in the `onto_example.rdf` ontology will be designated as `self`, while the information imported from the `lotr_example.owl` ontology will be marked using the imported ontology's IRI. The latter makes it possible to import the relevant ontology when converting the AgentSpeak code back to an ontology. Since we want the information that is transferred into AgentSpeak code to be complete, the various concepts of the imported ontology are included in the output.

In [ ]:
from bowldi import BOWLDIConverter

converter = BOWLDIConverter(
    input_data_path="onto_example.rdf",
)

if converter.get_response().get("status") == "success":
    print(converter.agentspeak_output)
else:
    print(converter.get_response())

If everything was as expected, the AgentSpeak output should be saved in the `output.asl` file and look like follows:

```prolog
concept(wizard)[source(['<path to ontology>/lotr_example.owl'])].
is_a(wizard, person)[source(['<path to ontology>/lotr_example.owl'])].
concept(human)[source(['<path to ontology>/lotr_example.owl'])].
is_a(human, person)[source(['<path to ontology>/lotr_example.owl'])].
concept(elf)[source(['<path to ontology>/lotr_example.owl'])].
is_a(elf, person)[source(['<path to ontology>/lotr_example.owl'])].
concept(kingdom)[source(['<path to ontology>/lotr_example.owl'])].
concept(person)[source(['<path to ontology>/lotr_example.owl'])].
concept(ring)[source(['<path to ontology>/lotr_example.owl'])].
concept(dwarf)[source(['<path to ontology>/lotr_example.owl'])].
is_a(dwarf, person)[source(['<path to ontology>/lotr_example.owl'])].
concept(hobbit)[source(['<path to ontology>/lotr_example.owl'])].
is_a(hobbit, person)[source(['<path to ontology>/lotr_example.owl'])].
object_property(person, is_friend_with, person)[source(['<path to ontology>/lotr_example.owl'])].
object_property(kingdom, has_king, person)[source(['<path to ontology>/lotr_example.owl'])].
object_property(person, is_in_team_with, person)[source(['<path to ontology>/lotr_example.owl'])].
object_property(person, has_ring, ring)[source(['<path to ontology>/lotr_example.owl'])].
object_property(person, fights_against, person)[source(['<path to ontology>/lotr_example.owl'])].
data_property(person, has_name, str)[source(['<path to ontology>/lotr_example.owl'])].
data_property(ring, description, str)[source(['<path to ontology>/lotr_example.owl'])].
individual(gandalf_the_grey, wizard)[source(['self'])].
individual(aragorn_son_of_arathorn, human)[source(['self'])].
individual(legolas, elf)[source(['self'])].
relation(gandalf_the_grey, has_name, "Gandalf the Grey")[source(Bogdan)].
relation(gandalf_the_grey, has_name, "Mithrandir")[source(self)].
relation(aragorn_son_of_arathorn, is_friend_with, gandalf_the_grey)[source(self)].
relation(aragorn_son_of_arathorn, is_friend_with, legolas)[source(self)].
relation(aragorn_son_of_arathorn, has_name, "Aragorn, son of Arathorn")[source(self)].
relation(legolas, has_name, "Legolas Greenleaf")[source(self)].
```

## Example 4: Converting AgentSpeak with external source to OWL

The `BOWLDIConverter` class can convert AgentSpeak code into an ontology file even when source argument of some beliefs references another source. If that other source is an ontology (i.e. if the source is a link it is assumed to be an ontology), then the translation process will try to import the ontology before processing the rest of the contents of the AgentSpeak code. The following is a simple example of reading the AgentSpeak code created in the previous example and converting it to an ontology file.

In [ ]:
from bowldi import BOWLDIConverter

converter = BOWLDIConverter(
    input_data_path="output.asl",
)

print(converter.get_response())

The output of the last line should be indicating success:

```json
{
  "status": "success",
  "message": "Conversion complete.",
  "file": "<path to file>/output.owl"
}
```

The rendered file should not contain definitions of the concepts defined in the imported ontology. Instead, only the concepts designated as not being sourced by an external ontology should be included.

One major difference between the initial ontolgoy of Example 3 above, and the output ontology of this example is that all the concepts that are "native" to the observed ontology (and not to the imported ontology) have one new property added, namely the `isDefinedBy` annotation property. The value of this annotation data property is sourced in the `source` argument of a specific belief. This property is used to indicate the source of the information, i.e. the agent that provided the information.

| Ejercicio 6 --- OPCIONAL |
|------------:|
| Modifica el ejemplo de `BOWLDI` para que sea un diálogo entre dos agentes `SPADE``.|

## Example 5: Loading rendered AgentSpeak code into SPADE

In [ ]:
import asyncio
import time
import spade
from spade_bdi.bdi import BDIAgent

a = BDIAgent("BasicAgent_BDI@localhost", "SPADE", "output.asl")

await a.start()
await asyncio.sleep(1)

a.bdi.print_beliefs()

time.sleep(1)
await a.stop()

